### Final collaborative Recommender

In [144]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors
from collections import Counter

import warnings
warnings.filterwarnings('ignore')


In [171]:
class Recommender:
    
    def __init__(self,profiles,user_activity,recent_activity,dataset):
        self.df = dataset
        self.profiles = profiles
        self.activity = user_activity 
        self.recent_activity = recent_activity
    
    def get_features(self,dataframe):
        #getting dummies of dataset
        nutrient_dummies = dataframe.Nutrient.str.get_dummies()
        disease_dummies = dataframe.Disease.str.get_dummies(sep=' ')
        diet_dummies = dataframe.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)
     
        return feature_df
    
    def find_neighbors(self,dataframe,features,k):
        features_df = self.get_features(dataframe)
        total_features = features_df.columns  
        d = dict()
        for i in total_features:
            d[i]= 0
        for i in features:
            d[i] = 1
        final_input = list(d.values())
        
        similar_neighbors = self.k_neighbor([final_input],features_df,dataframe,k)
        return similar_neighbors
    
    def k_neighbor(self,inputs,feature_df,dataframe,k):
        
        #initializing model with k neighbors
        model = NearestNeighbors(n_neighbors=k,algorithm='ball_tree')
        
        # fitting model with dataset features
        model.fit(feature_df)
        
        df_results = pd.DataFrame(columns=list(dataframe.columns))
        
        # getting distance and indices for k nearest neighbor
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results = df_results.append(dataframe.loc[i])

        df_results = df_results.reset_index(drop=True)
        return df_results
    
    def user_based(self,features,user_id):

        similar_users = self.find_neighbors(self.profiles,features,10)
        users = list(similar_users.User_Id)
        results = self.activity[self.activity.User_Id.isin(users)] #taking acitivies
        results = results[results['User_Id']!=user_id] # selecting those which are not reviewed by user
        results = results.filter(['Meal_Id','Name','Nutrient','Veg_Non','description','Price','Review'])

        results = results.drop_duplicates(subset=['Name'])
        results = results.reset_index(drop=True)
        return results
        
    def recent_activity_based(self):
        meal_ids = list(self.recent_activity.Meal_Id.unique())
        recent_data = self.df[self.df.Meal_Id.isin(meal_ids)][['Nutrient','catagory','Disease','Diet']].reset_index(drop=True)

        disease = []
        diet = []
        for i in range(recent_data.shape[0]):
            for j in recent_data.loc[i,'Disease'].split():
                disease.append(j)
        for i in range(recent_data.shape[0]):
            for j in recent_data.loc[i,'Diet'].split():
                diet.append(j)
                
        value_counts = recent_data.Nutrient.value_counts()
        m = recent_data.Nutrient.value_counts().mean()
        features = list(value_counts[recent_data.Nutrient.value_counts()>m].index)
        a = dict(Counter(disease))
        
        m = np.mean(list(a.values()))
        for i in a.items():
            if i[1]>m:
                features.append(i[0])
        a = dict(Counter(diet))
        m = np.mean(list(a.values()))
        for i in a.items():
            if i[1]>m:
                features.append(i[0])
                
        similar_neighbors = self.find_neighbors(self.df,features,10)
        return similar_neighbors.filter(['Meal_Id','Name','Nutrient','Veg_Non','description','Price','Review'])
        
    def recommend(self,user_id):
        #finding user's profile features by id
        profile = self.profiles[self.profiles['User_Id']==user_id]
        features = []
        features.append(profile['Nutrient'].values[0])
        features.extend(profile['Disease'].values[0].split())
        features.extend(profile['Diet'].values[0].split())
        df1 = self.user_based(features,user_id)
        df2 = self.recent_activity_based()
        df = pd.concat([df1,df2])
      
        df = df.drop_duplicates('description').reset_index(drop=True)
        return df

In [172]:
user_id = 'User_71'  # user id of current user
profiles = pd.read_csv('user_Profiles.csv') # profiles of all users
user_activity = pd.read_csv('user_activity.csv') # activities of all users (meals eaten / purchased)
recent_activity = pd.read_csv('recent_activity.csv') # recent activities of current user (meals liked,rated,searched)
dataset = pd.read_csv('dataset.csv') # main dataset


ob = Recommender(profiles,user_activity,recent_activity,dataset)
result = ob.recommend(user_id)
result

,Meal_Id,Name,Nutrient,Veg_Non,description,Price,Review
0,meal_id6,japanese curry arancini with barley salsa,calcium,veg,"japanese curry, sticky rice, cheese inside ric...",420,1
1,meal_id15,almond and cranberry poha,iron,veg,"almond flakes, onion, poha, cranberries (froze...",235,9
2,meal_id20,baked multigrain murukku,iron,veg,"oats, ragi flour (bhakri atta), wheat flour, r...",220,1
3,meal_id22,baked namak para,magnesium,veg,"whole wheat flour (atta), refined flour (maida...",360,5
4,meal_id31,baked almond kofta,calcium,veg,"potato (boiled), nutmeg, milk, almonds (crushe...",450,3
5,meal_id49,christmas tree pizza,chloride,veg,"pizza dough (2 boules), red pepper, red onion,...",325,1
6,meal_id53,chocolate appo,magnesium,veg,"rice, coconut, baking powder, vanilla extract,...",505,5
7,meal_id60,caramelized sesame smoked almonds,calcium,veg,"साबुत बादाम, सफेद और काले तिल, चीनी, रोजमेरी, ...",255,5
8,meal_id33,moong dal kiwi coconut soup,sodium,veg,"धूली मूंग दाल नमक के साथ उबली हुई, कीवी, नारिय...",235,8
9,meal_id69,banana and maple ice lollies,selenium,veg,"केले, ग्रीक योगर्ट, हैवी क्रीम, प्योर मैपल सिर...",505,8
